This notebook illustrate how to connect to remote service.

## `Aletheia` service

`aletheia` service provides nice interface to read/write memory systems composed of hyper-vectors.

Currently it provides the following operations: 

* `get`: get/retrieve chunks that meets certain criteria;

* `set`: set/write chunks with certain metadata;

The chunks can be terminal, representing non-divisible concepts.
In addition, we can produce composite chunk that are sequences of chunks (terminal or composite), set of chunks, etc.

For example, if we model the English alphabet as terminal chunks, then each English word can be modelled as sequence of alphabets.

NOTE this requires `alatheia` service, either running at local machine or an Internet-accessible IP.


In [9]:
import grpc
from kongming.util import remote_memory
from kongming.data import lang

from kongming.api import aletheia_pb2, chunk_pb2, memory_pb2 

with grpc.insecure_channel(target="localhost:8080") as channel:
    m = remote_memory.RemoteMemory(channel)
    lang.populate_ascii(m)
    lang.populate_words(m, "lang:en", [
        "the", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog",
        "world", "catherine", "university", "rate", "kitchen",
        "something", "activity", "necessary", "other", "stuff",
        "pattern", "director", "dying", "chinese",
        "specialize", "some", "for", "cat",
        "relax", "ing", "lifetime",
        "uzbekistan",
        "statistical"])

The script above connects to `aletheia` service, write English alphabets, followed by constructing some English words (as sequences of alphabets).

Some simple queries:

In [10]:
with grpc.insecure_channel(target="localhost:8080") as channel:
    m = remote_memory.RemoteMemory(channel)
    got = m.get(memory_pb2.ChunkPicker(
        ref=chunk_pb2.ChunkReference(namespace="lang:en", token="relax")),
        session=aletheia_pb2.Session(keep_genesis=True, keep_semantics=True))
    print(got)

[elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 16360574669867971005
  sparse_constrained {
    seed: 11501770689536962977
  }
}
category: SEQUENCE
genesis {
  sequence {
    members {
      ref {
        namespace: "unicode"
        token: "r"
      }
    }
    members {
      ref {
        namespace: "unicode"
        token: "e"
      }
    }
    members {
      ref {
        namespace: "unicode"
        token: "l"
      }
    }
    members {
      ref {
        namespace: "unicode"
        token: "a"
      }
    }
    members {
      ref {
        namespace: "unicode"
        token: "x"
      }
    }
  }
}
refs {
  namespace: "lang:en"
  token: "relax"
}
semantic {
  hint: KNOT
  model: MODEL_1M_10BIT
  stable_hash: 7666327263937266486
  knot {
    parts {
      hint: SPARSE_CONSTRAINED
      sparse_constrained {
        prewired: PREWIRED_SEQUENCE_START
      }
    }
    parts {
      hint: NECKLACE
      necklace {
        seed: 11501770689536962977


The output explains how the sequence of `relax` were constructed in the memory.
First it's a `SEQUENCE` made of `r`, `e`, `l`, `a`, and `x` from namespace of `unicode`.
Its semantic code is the combination of algebraic operation of both binding (knot) and bundling (necklace).

Next example illustrates how we can recover individual member from a composite vector.

In [11]:
with grpc.insecure_channel(target="localhost:8080") as channel:
    m = remote_memory.RemoteMemory(channel)
    wordPicker = memory_pb2.ChunkPicker(
        ref=chunk_pb2.ChunkReference(
            namespace="lang:en",
            token="pattern"))
    got = m.get(memory_pb2.ChunkPicker(
        sequence_members_of=memory_pb2.SequenceMembersOfPicker(composite=wordPicker)))
    print(got)

[elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 1614520157068643625
  sparse_constrained {
    seed: 3435508721523563227
  }
}
category: TERMINAL
refs {
  namespace: "unicode"
  token: "p"
}
weight: 150
, elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 429238972617998172
  sparse_constrained {
    seed: 14963882848117934959
  }
}
category: TERMINAL
refs {
  namespace: "unicode"
  token: "a"
}
weight: 153
, elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 3175895842260726240
  sparse_constrained {
    seed: 2212033975698694196
  }
}
category: TERMINAL
refs {
  namespace: "unicode"
  token: "t"
}
weight: 137
, elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 3175895842260726240
  sparse_constrained {
    seed: 2212033975698694196
  }
}
category: TERMINAL
refs {
  namespace: "unicode"
  token: "t"
}
weight: 149
, elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10

In this case, with a simple composite code for word `pattern`, the memory can recover its members: `p`, `a`, `t`, `t`, `e`, `r`, `n`, with each taking a weight/overlap of roughly 1/7 of the total 1024 ON bits.

Fuzzy query can be performed like:

In [12]:
with grpc.insecure_channel(target="localhost:8080") as channel:
    m = remote_memory.RemoteMemory(channel)
    wordPicker = memory_pb2.ChunkPicker(
        ref=chunk_pb2.ChunkReference(
            namespace="lang:en",
            token="world"))
    got = m.get(memory_pb2.ChunkPicker(near_neighbor=memory_pb2.NearNeighbor(target=wordPicker)))
    print(got)

[elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 4457954920353104723
  sparse_constrained {
    seed: 14336416972779868589
  }
}
category: SEQUENCE
refs {
  namespace: "lang:en"
  token: "world"
}
weight: 1024
, elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 4334755355221515437
  sparse_constrained {
    seed: 13896323908361552193
  }
}
category: SEQUENCE
refs {
  namespace: "lang:en"
  token: "for"
}
weight: 148
, elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 2914648003062445619
  sparse_constrained {
    seed: 3401615469385435847
  }
}
category: SEQUENCE
refs {
  namespace: "lang:en"
  token: "fox"
}
weight: 81
, elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 5239658206539774533
  sparse_constrained {
    seed: 15671600646850579412
  }
}
category: SEQUENCE
refs {
  namespace: "lang:en"
  token: "dog"
}
weight: 68
, elemental {
  hint: SPARSE_CONSTRAINED
  model:

This result is worth careful examination and discussion.

* First of all, the word `world` itself is returned (as full match, overlap=1024);
* the word `for` is returned as second, with weight/overlap=148, since there are two letters overlap: `o` and `r`;
* the word `fox` is returned, with weight/overlap=81, 1 letter overlapping: `o`;
* the word `dog` is returned, with weight/overlap=68, 1 letter overlapping: `o`;
* the word `some` is returned, with weight/overlap=46, 1 letter overlapping: `o`;
* the word `director` is returned, with weight/overlap=29, 1 letter overlapping: `r`;
* the word `something` is returned, with weight/overlap=24, 1 letter overlappng: `o`.

All fuzzy matching is done at the server side, modelling cognitive similarity with the query word.
Currently the matching can be achieved within a few milli-seconds.

Similarly, spelling suggestions (for other words) also work perfectly.